In [1]:
import Tiled_Classification_RF as TCRF
import os, tempfile
from osgeo import gdal, ogr, gdal_array # I/O image data
import numpy as np # math and array handling
import matplotlib.pyplot as plt # plot figures
from sklearn.ensemble import RandomForestClassifier # classifier
import pandas as pd # handling large data as table sheets
import geopandas as gpd # handling large data as shapefiles
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix  # calculating measures for accuracy assessment
import datetime
from xgboost import XGBClassifier
# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()
from GIStools.GIStools import preprocess_SfM_inputs
from GIStools.Stitch_Rasters import stitch_rasters
from GIStools.Grid_Creation import create_grid
from GIStools.Raster_Matching import pad_rasters_to_largest
from input_parameters import InputParameters
import joblib

In [2]:
#-------------------Required User Defined Inputs-------------------#
params = InputParameters()

DEM_path = params.DEM_path
ortho_path = params.ortho_path
output_folder = params.output_folder
model_path = params.model_path
training_path = params.training_path
validation_path = params.validation_path
attribute = params.attribute
validation_path_2 = params.validation_path_2
grid_ids = params.grid_ids
grid_path = params.grid_path
process_training_only = params.process_training_only
est = params.est
n_cores = params.n_cores
gradient_boosting = params.gradient_boosting
verbose = params.verbose
stitch = params.stitch

#--------------------Input Preparation-----------------------------#
#Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

#List of grid-clipped images to classify and associated id values
in_dir = os.path.join(output_folder, 'RF_Tiled_Inputs')
#output folder for list of img_path_list grid-clipped classified images

# directory, where the classification image should be saved:
results_folder = os.path.join(output_folder, 'RF_Results')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
classification_image = os.path.join(results_folder, 'Classified_Training_Image.tif')
classified_tile_folder = os.path.join(results_folder, 'Classified_Tiles')
if not os.path.exists(classified_tile_folder) and process_training_only == False:
    os.makedirs(os.path.join(classified_tile_folder))

results_txt = os.path.join(output_folder, 'Results_Summary.txt') # directory, where the all meta results will be saved

In [ ]:
#==================== Preprocessing ====================#
    #Create grid cells to process large rasters in chunks. 
#Each grid cell is the size of the extent training and validation shapefiles
if grid_path is None:
    train_val_grid_id, grid_path, _ = create_grid([training_path,validation_path], DEM_path, in_dir)
    train_tile_path = os.path.join(in_dir, f'stacked_bands_tile_input_{train_val_grid_id}.tif') # grid-clipped-image containing the training data
    if process_training_only: #preprocess_function will now only process the training tile
        grid_ids.append(train_val_grid_id)
        params.grid_ids = grid_ids
    print('Training Grid ID: {}'.format(train_val_grid_id)) 
else:
    grid = gpd.read_file(grid_path)
    params.grid_ids = grid['ID'].values.tolist() if grid_ids is None else grid_ids
    print('Grid IDs: {}'.format(grid_ids))  
#Bands output from preprocess function: Roughness, R, G, B, Saturation, Excessive Green Index
grid_ids, input_dict = preprocess_SfM_inputs(grid_path, ortho_path, DEM_path, params.grid_ids, in_dir, verbose=verbose) #Prepare input stacked rasters for random forest classification
print('Grid IDs to process: {}'.format(grid_ids))

img_path_list, id_values = TCRF.find_files(in_dir) # list of all grid-clipped images to classify and associated id values
attribute_names = TCRF.print_attributes(training_path) # print the attributes in the training shapefile
#===========================Main Classification Loop===========================#

TCRF.print_header_params(results_txt, params) # print the header for the results text file

In [5]:
import multiprocessing
from pathlib import Path


# Function to process a single tile
def process_tile(img_path, id_value, rf_model, output_dir, results_file):
    print(f"Processing {img_path}, ID value: {id_value}")
    current_tile, current_tile_3Darray = TCRF.extract_image_data(img_path, results_file, log=False)
    current_tile_2Darray = TCRF.flatten_raster_bands(current_tile_3Darray)
    current_class_prediction = TCRF.predict_classification(rf_model, current_tile_2Darray, current_tile_3Darray)
    current_masked_prediction = TCRF.reshape_and_mask_prediction(current_class_prediction, current_tile_3Darray)
    output_file = output_dir / f"Classified_Tile_{id_value}.tif"
    TCRF.save_classification_image(output_file, current_tile, current_tile_3Darray, current_masked_prediction)
    print(f'Tile saved to: {output_file}')
    # Clean up
    del current_class_prediction, current_masked_prediction, current_tile_3Darray, current_tile_2Darray

# Function to process all tiles in bulk using multiprocessing
def bulk_process(tiles, rf_model, classified_tile_folder, results_txt):
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        pool.starmap(process_tile, [(img_path, id_value, rf_model, classified_tile_folder, results_txt) for img_path, id_value in tiles])


# Load the saved model from the path
model_path = r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\LM2\LM2_2023___070923 - XGB Saved Model\RF_Model.joblib"
model = joblib.load(model_path)  # Load the model
in_dir = r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\LM2 - 081222 Full Run\RF_Tiled_Inputs"
classified_tile_folder_path = Path(classified_tile_folder)
img_path_list, id_values = TCRF.find_files(in_dir)  # Get the list of tiles to process
tiles = zip(img_path_list, id_values)
bulk_process(tiles, model, classified_tile_folder_path, results_txt)
